# Connect to ChatGPT API
In this lesson, you'll manage to connect to the OpenAi and AzureOpenAI APIS for ChatGPT.

## Setup
#### Load the API key and relevant Python libraries.

In [0]:
!pip install -qU openai
dbutils.library.restartPython()

In [0]:
import os
import openai
from openai import OpenAI
from openai import AzureOpenAI

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#openai.api_base = "https://chatgpt-summarization.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.environ["OPENAI_API_KEY"]

openai_deploy_name = "model-gpt-35-turbo"
openai_model_name = "gpt-35-turbo"

#### OpenAI or AzureOpenAI?  

When you access the model via the API in Azure OpenAI you will need to refer to the deployment name rather than the underlying model name in API calls. This is one of the key differences between OpenAI and Azure OpenAI. OpenAI only requires the model name, Azure OpenAI always requires deployment name, even when using the model parameter. In our docs we often have examples where deployment names are represented as identical to model names to help indicate which model works with a particular API endpoint. Ultimately your deployment names can follow whatever naming convention is best for your use case.

#### OpenAI:  
```
client = OpenAI(api_key=openai.api_key)

completion = client.completions.create(model="<model_name>", prompt="<prompt>")  
chat_completion = client.chat.completions.create(model="<model_name>",messages="<messages>")  
embedding = client.embeddings.create(model="<model_name>", input="<input>")  
```

#### AzureOpenAI:
```
client = AzureOpenAI(api_key=openai.api_key, api_version=openai.api_version, azure_endpoint=openai.api_base)  

completion = client.completions.create(model="<model_deployment_name>", prompt="<prompt>")  
chat_completion = client.chat.completions.create(model="<model_deployment_name>", messages="<messages>")  
embedding = client.embeddings.create(model="<model_deployment_name>", input="<input>")  
```

In [0]:
client = AzureOpenAI(api_key=openai.api_key,
                     api_version=openai.api_version,
                     azure_endpoint=openai.api_base,
                     )

### Simple Completions  

OpenAI refers to text generation simply as completions as in text completion. It is interesting to note the naming convention, which derives from how these language models generate text via the use of word probability, one word at a time as it completes the initial starting words to form complete sentences. Completions models are being discontinued (text-davinci-003, text-davinci-002, davinci, curie, babbage, ada)  

In [0]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

completion = client.completions.create(model=openai_deploy_name,
                                       prompt=prompt,
                                       temperature=0,
                                       max_tokens=800,
                                       #max_tokens=OPENAI_MAX_TOKENS[openai_model_name],
                                       top_p=1,
                                       frequency_penalty=0,
                                       presence_penalty=0,
                                       stop=None,
                                       )

In [0]:
#print(completion.model_dump_json(indent=2))
print(completion.choices[0].text)

We can see that the response has generated much more than a simple answer, but continued the probability-based text generation.  
Let's suppose we were using one of the (soon to be deprecated) fine tuned completion models (**model-text-davinci-003**):

In [0]:
completion2 = client.completions.create(model="model-text-davinci-003",
                                        prompt=prompt,
                                        temperature=0,
                                        max_tokens=800,
                                        #max_tokens=OPENAI_MAX_TOKENS[openai_model_name],
                                        top_p=1,
                                        frequency_penalty=0,
                                        presence_penalty=0,
                                        stop=None,
                                        )

print(completion2.choices[0].text)

We can see that the response is very different from just predicting next sentences.  
But as these models are being discontinued, we should focus on Chat Completions.  

### Chat Completions  

An alternative to completions are chat completions that are GPT models that have been optimized for conversational text (gpt-4, gpt-3.5-turbo). We are probably most familiar with this GPT type as the underlying GPT 3.5 and their flagship GPT 4 are powering the vastly popular ChatGPT. An added benefit of chat completions is that they are less prone to prompt injection attacks as user-provided content is separate from instruction prompt.  

Within the OpenAI API, messages often adopt [specific roles](https://www.tinydesk.ai/post/unlocking-the-potential-understanding-the-roles-in-chatgpt-s-api-system-user-assistant-and) to guide the model’s responses. Commonly used roles include “system,” “user,” and “assistant.”  
+ The “system” provides high-level instructions
+ The “user” presents queries or prompts  
+ The “assistant” is the model’s response.  

By differentiating these roles, we can set the context and direct the conversation efficiently.  
There is also the role "function", that we will present later.

##### Example #1    
(reusing the prompt)

In [0]:
chat_completion = client.chat.completions.create(
    model=openai_deploy_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{prompt}"},
    ],
)

In [0]:
print(chat_completion.choices[0].message.content)

##### Example #2  
(Adding some history of conversations to the request)  

In [0]:
chat_completion = client.chat.completions.create(
    model=openai_deploy_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
    ],
)

In [0]:
#print(chat_completion.model_dump_json(indent=2))
print(chat_completion.choices[0].message.content)

### Wraping in a function:  

We could encapsulate all the commands in one function, as seen below. This could be customized to any specific needs.

In [0]:
def ask(prompt):
    try:
        chat_completion = client.chat.completions.create(
            model=openai_deploy_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ],
            )
        print(chat_completion.choices[0].message.content)

    except openai.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
    except openai.error.AuthenticationError as e:
        print(f"OpenAI API returned an Authentication Error: {e}")
    except openai.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
    except openai.error.InvalidRequestError as e:
        print(f"Invalid Request Error: {e}")
    except openai.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
    except openai.error.ServiceUnavailableError as e:
        print(f"Service Unavailable: {e}")
    except openai.error.Timeout as e:
        print(f"Request timed out: {e}")
    except:
        print("An exception has occured.")

In [0]:
ask(prompt=prompt)